# Imports and data loading

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


### Imports

In [22]:
exec(open("props.py"))
exec(open("helpers.py"))
from __future__ import division
output_notebook(); #force plots to display inline
# %matplotlib inline

Loading BokehJS ...

### Load data (only do once and then proceed with analysis!)

In [3]:
# make sure we are using an updated copy of the file, since jupyter tends to cache notebooks
%load_ext autoreload
%autoreload loadData

# takes about 4 mins
%run loadData.ipynb

There are 
884
fake ages 

There are
192432
previously unknown ages 

Total unknown ages is 
193316
out of 
1731594
total ages
for percentage
0
There are 
0
fake ages 

There are
193316
previously unknown ages 

Total unknown ages is 
193316
out of 
1731594
total ages
for percentage
0


# Overview analysis plots

### Generic names so I can refactor here instead of refactoring in each plot

In [51]:
data = Jun17
# del Jun17 # this might be necessary if memory in the browser becomes an issue
month = "June"
numDaysMonth = 30

abbrDayLabels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
dayLabels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

## Total trips taken

In [52]:
data.shape[0]

1731594

## Trips per day of month

In [53]:
# check if any nans
data.dayOfMonth[data.dayOfMonth.isnull()].shape

(0L,)

In [54]:
monthLabels = []
for day in range(numDaysMonth):
    monthLabels.append(month + " " + str(day+1))
# monthLabels

In [55]:
p = figure(x_range = monthLabels)
standardize(p)
standardizevbar(p)

x = monthLabels
top = data.groupby('dayOfMonth').count().max(1).values
width = 0.7
bottom = 0
p.vbar(x=x,top=top,width=width,bottom=bottom, color='skyblue');

p.title.text = "Trips Per Day of Month"
p.xaxis.axis_label = "day of month"
p.yaxis.axis_label = "# of trips"

In [56]:
show(p)

## Trips per day of week

In [57]:
# check if any nans
data.dayOfWeek[data.dayOfWeek.isnull()].shape

(0L,)

In [58]:
p = figure(x_range = dayLabels)
standardize(p)
standardizevbar(p)

x = dayLabels # Monday=0, Sunday=6
temp = data.groupby('dayOfWeek').count().max(1).values
temp[3] = temp[3] - temp[3]/5 #Have to take off a portion of Thurs/Fri since Jun 2017 had 5 T/F and 4 of the other days
temp[4] = temp[4] - temp[4]/5
top = temp
width = 0.85
bottom = 0
p.vbar(x=x,top=top,width=width,bottom=bottom, color='skyblue');

# add labels to the bars
addPctsToBars(p, x, top, data.shape[0]-temp[3]/5-temp[4]/5)

p.title.text = "Trips Per Day of Week"
p.xaxis.axis_label = "day of week"
p.yaxis.axis_label = "# of trips"

p.left[0].formatter.use_scientific = False
# p.xaxis.major_label_orientation = 0
# p.xaxis.major_label_standoff = 10

98.46


In [59]:
show(p)

## Trips per type of user

In [17]:
# check if any nans
data.usertype[data.usertype.isnull()].shape

(0L,)

In [23]:
userLabels = ["Subscriber","Temporary Pass"]

p = figure(x_range = userLabels)
standardize(p)
standardizevbar(p)

x = userLabels # subscriber=1, temp=2
top = data.groupby('usertype').count().max(1).values
width = 0.85
bottom = 0
p.vbar(x=x,top=top,width=width,bottom=bottom, color='skyblue');

addPctsToBars(p, x, top, data.shape[0])

p.title.text = "Trips Per Type Of User"
p.xaxis.axis_label = "type of user"
p.yaxis.axis_label = "# of trips"

p.left[0].formatter.use_scientific = False
# p.xaxis.major_label_orientation = 0
# p.xaxis.major_label_standoff = 10

100.0


In [24]:
show(p)

## Trips by gender

In [25]:
genderLabels = ["Unknown", "Male", "Female"]

p = figure(x_range = genderLabels)
standardize(p)
standardizevbar(p)

x = genderLabels # unknown=0, male=1, female=2
top = data.groupby('gender').count().max(1).values
width = 0.85
bottom = 0
p.vbar(x=x,top=top,width=width,bottom=bottom, color='skyblue');

addPctsToBars(p, x, top, data.shape[0])

p.title.text = "Trips Per Gender of User"
p.xaxis.axis_label = "gender of user"
p.yaxis.axis_label = "# of trips"

p.left[0].formatter.use_scientific = False
# p.xaxis.major_label_orientation = 0
# p.xaxis.major_label_standoff = 10

100.0


In [26]:
show(p)

## Trip duration info

In [27]:
temp = data.tripduration/60

counts = np.empty(7)
counts[0] = temp[(temp>=0) & (temp<5)].size
counts[1] = temp[(temp>=5) & (temp<10)].size
counts[2] = temp[(temp>=10) & (temp<15)].size
counts[3] = temp[(temp>=15) & (temp<20)].size
counts[4] = temp[(temp>=20) & (temp<25)].size
counts[5] = temp[(temp>=25) & (temp<30)].size
counts[6] = temp[(temp>=30)].size

In [28]:
durationLabels = ["0-5","5-10","10-15","15-20","20-25","25-30","30+"]

p = figure(x_range = durationLabels)
standardize(p)
standardizevbar(p)

x = durationLabels
top = counts
width = 0.85
bottom = 0
p.vbar(x=x,top=top,bottom=bottom,width=width,color='skyblue');

addPctsToBars(p, x, top, data.shape[0])

p.title.text = "Trips Per Trip Duration"
p.xaxis.axis_label = "trip duration (minutes)"
p.yaxis.axis_label = "# of trips"

p.left[0].formatter.use_scientific = False
# p.xaxis.major_label_orientation = 0
# p.xaxis.major_label_standoff = 10

100.0


In [29]:
show(p)

## Trips taken by time of day

In [30]:
hourLabels = []
for hour in range(24):
    if (hour>=0 and hour<=9):
        hourLabels.append("0"+str(hour))
    else:
        hourLabels.append(str(hour))
# hourLabels

In [31]:
start = data.groupby('startHour').count().max(1).values
stop = data.groupby('stopHour').count().max(1).values

mins = np.minimum(start,stop)
maxes = np.maximum(start,stop)

### Mins and maxes technique

In [32]:
# p = figure(x_range = hourLabels)
# p.plot_width=900
# standardize(p)
# standardizevbar(p)

# x = hourLabels
# top = maxes
# bottom = 0
# width=0.7
# p.vbar(x=x,top=top,width=width,bottom=bottom, color='hotpink')

# startPcts = start / data.shape[0] * 100
# startPcts = np.round(startPcts,1)
# startPctsStrings = []
# for i in range(startPcts.size):
#     startPctsStrings.append(str(startPcts[i]) + "% start")

# startSource = ColumnDataSource(data=dict(x=x,
#                                     top=maxes,
#                                     pcts = startPctsStrings))
# labels = LabelSet(x='x',y='top',text='pcts',
#                   level='glyph',x_offset=0,y_offset=13,
#                   source=startSource,render_mode='canvas',
#                   text_align='center',text_font_size="7pt",angle=-0.75)
# p.add_layout(labels)
# print(sum(startPcts))

# stopPcts = stop / data.shape[0] * 100
# stopPcts = np.round(stopPcts,1)
# stopPctsStrings = []
# for i in range(stopPcts.size):
#     stopPctsStrings.append(str(stopPcts[i]) + "% stop")

# stopSource = ColumnDataSource(data=dict(x=x,
#                                     top=maxes,
#                                     pcts = stopPctsStrings))
# labels = LabelSet(x='x',y='top',text='pcts',
#                   level='glyph',x_offset=0,y_offset=30,
#                   source=stopSource,render_mode='canvas',
#                   text_align='center',text_font_size="7pt",angle=-.75)
# p.add_layout(labels)
# print(sum(stopPcts))

# top=mins
# p.vbar(x=x,top=top,width=width,bottom=bottom, color='skyblue')

# p.title.text = "Trips Per Hour of Day"
# p.xaxis.axis_label = "hour of day (24 hr)"
# p.yaxis.axis_label = "# of trips"

# p.left[0].formatter.use_scientific = False
# # p.xaxis.major_label_orientation = 0
# # p.xaxis.major_label_standoff = 10

In [33]:
# show(p)

### start and stop separated technique

In [34]:
p = figure(x_range = hourLabels)
p.plot_width=900
standardize(p)
standardizevbar(p)

x = hourLabels
top = start
bottom = 0
width=0.7
p.vbar(x=x,top=top,width=width,bottom=bottom, color='salmon',legend='start')

startPcts = start / data.shape[0] * 100
startPcts = np.round(startPcts,1)
startPctsStrings = []
for i in range(startPcts.size):
    startPctsStrings.append(str(startPcts[i]) + "% start")

startSource = ColumnDataSource(data=dict(x=x,
                                    top=maxes,
                                    pcts = startPctsStrings))
labels = LabelSet(x='x',y='top',text='pcts',
                  level='glyph',x_offset=0,y_offset=13,
                  source=startSource,render_mode='canvas',
                  text_align='center',text_font_size="7pt",angle=-0.75)
p.add_layout(labels)
print(sum(startPcts))

stopPcts = stop / data.shape[0] * 100
stopPcts = np.round(stopPcts,1)
stopPctsStrings = []
for i in range(stopPcts.size):
    stopPctsStrings.append(str(stopPcts[i]) + "% stop")

stopSource = ColumnDataSource(data=dict(x=x,
                                    top=maxes,
                                    pcts = stopPctsStrings))
labels = LabelSet(x='x',y='top',text='pcts',
                  level='glyph',x_offset=0,y_offset=30,
                  source=stopSource,render_mode='canvas',
                  text_align='center',text_font_size="7pt",angle=-.75)
p.add_layout(labels)
print(sum(stopPcts))

top=stop
p.vbar(x=x,top=top,width=width,bottom=bottom, color='skyblue',legend='stop',fill_alpha=0.75)

p.title.text = "Trips Per Hour of Day"
p.xaxis.axis_label = "hour of day (24 hr)"
p.yaxis.axis_label = "# of trips"

p.left[0].formatter.use_scientific = False
# p.xaxis.major_label_orientation = 0
# p.xaxis.major_label_standoff = 10

100.0
100.2


In [35]:
show(p)

In [37]:
# p = figure(x_range = hourLabels)
# # standardize(p)
# # standardizevbar(p)

# source = ColumnDataSource(data=dict(x=hourLabels,
#                                     start=data.groupby('startHour').count().max(1).values,
#                                     stop=data.groupby('stopHour').count().max(1).values),
                                    
#                          )
# p.vbar_stack(['start','stop'],x='x',color=['skyblue','orange'],source=source)
# show(p)

In [38]:
# # p = figure(x_range = hourLabels)
# standardize(p)
# standardizevbar(p)

# x = hourLabels
# top = data.groupby('startHour').count().max(1).values
# width = 0.85
# bottom = 0
# p.vbar_stack(["blah1"],x=x,top=top,width=width,bottom=bottom, color='skyblue');
# p.vbar_stack(["blah2"],x=x,top=data.groupby('stopHour').count().max(1).values,width=width,bottom=bottom,color='orange')

# addPctsToBars(p, x, top, data.shape[0])

# p.title.text = "Trips Per Hour of Day"
# p.xaxis.axis_label = "hour of day (24 hr)"
# p.yaxis.axis_label = "# of trips"

# p.left[0].formatter.use_scientific = False
# # p.xaxis.major_label_orientation = 0
# # p.xaxis.major_label_standoff = 10

# # TODO break into weekend vs weekday?

In [39]:
# blah1 = data.groupby('startHour').count().max(1).values
# blah2=data.groupby('stopHour').count().max(1).values
# myData = {'hours':hourLabels,
#         'start':blah1,
#         'stop':blah2
#        }
# source = ColumnDataSource(data=myData)
# p=figure(x_range=hourLabels)
# p.vbar_stack(['start','stop'],x='hours',color=['orange','blue'],source=source)

## Trips taken by birth year (age)

In [47]:
temp = data.age

counts = np.empty(8)
counts[0] = temp[temp.isnull()==True].shape[0]
counts[1] = temp[(temp>=0) & (temp<15)].size
counts[2] = temp[(temp>=15) & (temp<25)].size
counts[3] = temp[(temp>=25) & (temp<35)].size
counts[4] = temp[(temp>=35) & (temp<45)].size
counts[5] = temp[(temp>=45) & (temp<55)].size
counts[6] = temp[(temp>=55) & (temp<65)].size
counts[7] = temp[(temp>=65)].size

In [48]:
ageLabels = ["unknown","<15","15-25","25-35","35-45","45-55","55-65","65+"]

p = figure(x_range = ageLabels)
standardize(p)
standardizevbar(p)

x = ageLabels
top = counts
width = 0.85
bottom = 0
p.vbar(x=x,top=top,width=width,bottom=bottom, color='skyblue');

addPctsToBars(p, x, top, data.shape[0])

p.title.text = "Trips Per Age Group"
p.xaxis.axis_label = "age (years)"
p.yaxis.axis_label = "# of trips"

p.left[0].formatter.use_scientific = False
# p.xaxis.major_label_orientation = 0
# p.xaxis.major_label_standoff = 10

99.99


In [49]:
show(p)

## Trips taken by birth year compared with time of day?  With trip duration? With day of week?

In [43]:
# TODO if it relevant for the rest of analysis...could be interesting to see how age affects the time of day you travel
# or how long you travel for...maybe could do learning on this??

## Further analysis here, too

#### Test color scheme

In [50]:
p = figure()
x = [1,2,3]
colors = ['mediumpurple',
'skyblue',
'salmon',
'hotpink',
'sandybrown',
'darkseagreen',
'cornflowerblue']
for i in range(7):
    p.line(x, x+np.ones(3)*i, color=colors[i], legend="blah"+str(i),line_width=2.5)
show(p)